Bert Jim Chang, ID: 2205274

# Python code for the game

In [1]:
import tkinter as tk
from tkinter import simpledialog, messagebox
import math
import copy

class PubGame:
    def __init__(self, master, num_glasses):
        self.master = master
        self.master.title("Pub Game")

        #number of glasses in the circle.
        self.num_glasses = num_glasses
        
        #each glass holds water; overflow happens when > 1.0 pint.
        self.glass_fill = [0.0] * self.num_glasses
        
        #ali must distribute exactly 0.5 pints each turn.
        self.ALI_AMOUNT = 0.5
        self.amount_distributed_this_turn = 0.0
        
        #current player: "Ali" or "Beth".
        self.current_player = "Ali"
        
        #for Beth's move: store index of first clicked glass.
        self.beth_first_choice = None
        
        #turn counter.
        self.turn_counter = 1
        
        #share mode variables.
        self.share_mode = False
        self.share_selection = []
        
        #create frames.
        self.top_frame = tk.Frame(self.master)
        self.top_frame.pack(side=tk.TOP, fill=tk.X)
        
        self.button_frame = tk.Frame(self.master)
        self.button_frame.pack(side=tk.TOP, fill=tk.X, pady=5)
        
        self.canvas = tk.Canvas(self.master, width=700, height=700, bg="white")
        self.canvas.pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)
        
        #bind canvas resizing event.
        self.canvas.bind("<Configure>", self.on_canvas_resize)
        
        #status label.
        self.status_label = tk.Label(self.top_frame, text="", font=("Arial", 14))
        self.status_label.pack(side=tk.LEFT, padx=10)
        
        #buttons.
        self.undo_button = tk.Button(self.button_frame, text="Undo", command=self.undo_move)
        self.undo_button.pack(side=tk.LEFT, padx=5)
        
        self.share_button = tk.Button(self.button_frame, text="Share", command=self.toggle_share_mode)
        self.share_button.pack(side=tk.LEFT, padx=5)
        
        self.confirm_share_button = tk.Button(self.button_frame, text="Confirm Share", command=self.confirm_share)
        self.confirm_share_button.pack(side=tk.LEFT, padx=5)
        
        #for storing the glass graphical objects (position, id's, etc.).
        self.glass_coords = []
        self.draw_glasses()
        
        #for undo: maintain a per-turn history (only current turn moves can be undone).
        self.turn_history = []
        self.push_state()  # initial state for current turn
        
        self.update_status()
    
    # ------------------- Canvas Resize and Drawing -------------------
    def on_canvas_resize(self, event):
        """Redraw glasses based on the current canvas dimensions."""
        self.draw_glasses()
    
    def draw_glasses(self):
        """Recalculate positions and sizes of the glasses based on the current canvas dimensions.
        The glass radius is set to 9% of the smaller dimension (min 25 px) and the text font scales accordingly."""
        self.canvas.delete("all")
        
        #get current canvas dimensions.
        width = self.canvas.winfo_width()
        height = self.canvas.winfo_height()
        cx, cy = width / 2, height / 2
        
        #set the radius of the circle on which glasses are placed.
        circle_radius = 0.8 * min(width, height) / 2
        
        #compute glass (circle) radius relative to canvas size (increased from 7% to 9%).
        glass_radius = max(25, 0.09 * min(width, height))
        #compute font size relative to glass radius.
        font_size = max(8, int(glass_radius * 0.4))
        
        angle_step = 2 * math.pi / self.num_glasses
        self.glass_coords = []
        for i in range(self.num_glasses):
            angle = i * angle_step
            x = cx + circle_radius * math.cos(angle)
            y = cy + circle_radius * math.sin(angle)
            oval_id = self.canvas.create_oval(x - glass_radius, y - glass_radius,
                                              x + glass_radius, y + glass_radius,
                                              fill="lightblue", outline="blue", width=2)
            text_id = self.canvas.create_text(x, y, text="0.0000",
                                              font=("Arial", font_size), fill="black")
            self.canvas.tag_bind(oval_id, "<Button-1>", lambda e, idx=i: self.on_glass_click(idx))
            self.canvas.tag_bind(text_id, "<Button-1>", lambda e, idx=i: self.on_glass_click(idx))
            self.glass_coords.append((x, y, oval_id, text_id))
        self.update_glass_labels()
    
    def update_glass_labels(self):
        for i, (_, _, oval_id, text_id) in enumerate(self.glass_coords):
            current_fill = self.glass_fill[i]
            self.canvas.itemconfig(text_id, text=f"{current_fill:.4f}")
            # Change fill to red if overflow (strictly > 1.0 pint).
            if current_fill > 1.0:
                self.canvas.itemconfig(oval_id, fill="red")
            else:
                # Highlight selections for share mode or Beth's move.
                if self.share_mode and i in self.share_selection:
                    self.canvas.itemconfig(oval_id, fill="yellow")
                elif self.current_player == "Beth" and self.beth_first_choice == i:
                    self.canvas.itemconfig(oval_id, fill="orange")
                else:
                    self.canvas.itemconfig(oval_id, fill="lightblue")
    
    # ------------------- Turn and Status Management -------------------
    def update_status(self):
        if self.current_player == "Ali":
            text = f"Turn {self.turn_counter}: Ali's turn (distribute 0.5 pints)"
        else:
            text = f"Turn {self.turn_counter}: Beth's turn (empty two adjacent glasses)"
        self.status_label.config(text=text, fg="black")
    
    def next_player(self):
        # Switch player and clear per-turn history.
        self.current_player = "Beth" if self.current_player == "Ali" else "Ali"
        self.turn_counter += 1
        self.amount_distributed_this_turn = 0.0
        self.beth_first_choice = None
        self.share_mode = False
        self.share_selection.clear()
        self.turn_history = []
        self.push_state()  # initial state for new turn.
        self.update_status()
        self.update_glass_labels()
    
    def check_for_overflow(self):
        for fill in self.glass_fill:
            if fill > 1.0:  # strictly greater than 1.0 pint triggers overflow.
                self.status_label.config(text="Ali made a glass overflow! Ali wins!", fg="red")
                self.canvas.unbind("<Button-1>")
                return True
        return False
    
    # ------------------- Share Mode -------------------
    def toggle_share_mode(self):
        self.share_mode = not self.share_mode
        if not self.share_mode:
            self.share_selection.clear()
        self.update_glass_labels()
    
    def confirm_share(self):
        if not self.share_mode:
            messagebox.showinfo("Share Mode", "You must be in share mode to confirm a share.")
            return
        if len(self.share_selection) == 0:
            messagebox.showinfo("Share Mode", "No glasses selected to share among.")
            return
        if self.current_player == "Ali":
            remaining = self.ALI_AMOUNT - self.amount_distributed_this_turn
        else:
            remaining = float('inf')
        fraction_str = simpledialog.askstring("Share Water",
                                                "Enter fraction to share (e.g. '1/4' or '0.25'):",
                                                parent=self.master)
        if fraction_str is None:
            return
        try:
            share_amount = self.parse_fraction(fraction_str)
        except:
            messagebox.showerror("Invalid Input", "Could not parse fraction. Use e.g. '1/4' or '0.25'.")
            return
        if share_amount < 0:
            messagebox.showerror("Invalid Input", "Cannot share a negative amount.")
            return
        if share_amount > remaining + 1e-9:
            messagebox.showerror("Invalid Input", f"Cannot exceed remaining {remaining:.4f} pints this turn.")
            return
        n = len(self.share_selection)
        if n == 0:
            return
        each_portion = share_amount / n
        self.push_state()
        for idx in self.share_selection:
            self.glass_fill[idx] += each_portion
        if self.current_player == "Ali":
            self.amount_distributed_this_turn += share_amount
        self.update_glass_labels()
        if self.check_for_overflow():
            return
        if self.current_player == "Ali" and abs(self.amount_distributed_this_turn - self.ALI_AMOUNT) < 1e-9:
            self.next_player()
        self.share_selection.clear()
        self.share_mode = False
        self.update_glass_labels()
    
    # ------------------- Click Handlers -------------------
    def on_glass_click(self, glass_index):
        # If in share mode, toggle selection.
        if self.share_mode:
            if glass_index in self.share_selection:
                self.share_selection.remove(glass_index)
            else:
                self.share_selection.append(glass_index)
            self.update_glass_labels()
            return

        # Process click based on current player.
        if self.current_player == "Ali":
            self.handle_ali_click(glass_index)
        else:
            self.handle_beth_click(glass_index)
    
    def handle_ali_click(self, glass_index):
        remaining = self.ALI_AMOUNT - self.amount_distributed_this_turn
        if remaining <= 1e-9:
            self.next_player()
            return
        fraction_str = simpledialog.askstring("Ali's Turn",
                                                f"Enter fraction to add to Glass {glass_index} (remaining {remaining:.4f}):",
                                                parent=self.master)
        if fraction_str is None:
            return
        try:
            add_amount = self.parse_fraction(fraction_str)
        except:
            messagebox.showerror("Invalid Input", "Could not parse fraction. Use e.g. '1/4' or '0.25'.")
            return
        if add_amount < 0:
            messagebox.showerror("Invalid Input", "Cannot add a negative amount.")
            return
        if add_amount > remaining + 1e-9:
            messagebox.showerror("Invalid Input", f"Cannot exceed remaining {remaining:.4f} pints this turn.")
            return
        self.push_state()
        self.glass_fill[glass_index] += add_amount
        self.amount_distributed_this_turn += add_amount
        self.update_glass_labels()
        if self.check_for_overflow():
            return
        if abs(self.amount_distributed_this_turn - self.ALI_AMOUNT) < 1e-9:
            self.next_player()
    
    def handle_beth_click(self, glass_index):
        # On Beth's turn, highlight the first clicked glass.
        if self.beth_first_choice is None:
            self.beth_first_choice = glass_index
            self.update_glass_labels()
        else:
            first_idx = self.beth_first_choice
            second_idx = glass_index
            self.beth_first_choice = None  # Clear after second click.
            if self.are_adjacent(first_idx, second_idx):
                self.push_state()
                self.glass_fill[first_idx] = 0.0
                self.glass_fill[second_idx] = 0.0
                self.update_glass_labels()
            else:
                messagebox.showinfo("Invalid Move", "These glasses are not adjacent. Try again.")
            self.next_player()
    
    def are_adjacent(self, i, j):
        if i == j:
            return False
        return (j == (i + 1) % self.num_glasses) or (j == (i - 1) % self.num_glasses)
    
    # ------------------- Undo / History (Current Turn Only) -------------------
    def push_state(self):
        state = {
            'glass_fill': copy.deepcopy(self.glass_fill),
            'amount_distributed_this_turn': self.amount_distributed_this_turn,
            'beth_first_choice': self.beth_first_choice,
            'share_mode': self.share_mode,
            'share_selection': copy.deepcopy(self.share_selection)
        }
        self.turn_history.append(state)
    
    def undo_move(self):
        if len(self.turn_history) <= 1:
            messagebox.showinfo("Undo", "No moves to undo in the current turn.")
            return
        self.turn_history.pop()
        last_state = self.turn_history[-1]
        self.glass_fill = copy.deepcopy(last_state['glass_fill'])
        self.amount_distributed_this_turn = last_state['amount_distributed_this_turn']
        self.beth_first_choice = last_state['beth_first_choice']
        self.share_mode = last_state['share_mode']
        self.share_selection = copy.deepcopy(last_state['share_selection'])
        self.update_glass_labels()
        self.update_status()
    
    # ------------------- Helper -------------------
    def parse_fraction(self, s):
        s = s.strip()
        if '/' in s:
            num, denom = s.split('/')
            return float(num) / float(denom)
        else:
            return float(s)

# ------------------- Setup Window -------------------
def start_game():
    try:
        n = int(spin_num_glasses.get())
        if n < 3:
            raise ValueError
    except ValueError:
        n = 6
    game_window = tk.Toplevel(root)
    PubGame(game_window, n)

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Pub Game Setup")
    setup_frame = tk.Frame(root)
    setup_frame.pack(padx=20, pady=20)
    tk.Label(setup_frame, text="Number of Glasses:").grid(row=0, column=0, padx=5, pady=5)
    spin_num_glasses = tk.Spinbox(setup_frame, from_=3, to=20, width=5)
    spin_num_glasses.grid(row=0, column=1, padx=5, pady=5)
    spin_num_glasses.delete(0, tk.END)
    spin_num_glasses.insert(0, "8")
    start_button = tk.Button(setup_frame, text="Start Game", command=start_game)
    start_button.grid(row=1, column=0, columnspan=2, pady=10)
    root.mainloop()


# Code for recurrence relation

In [4]:
f1 = 1/6

iterations = 100

fk_values = [f1]

for k in range(1, iterations):
    fk_next = (fk_values[-1] + 0.5) / 3
    fk_values.append(fk_next)

for i, fk in enumerate(fk_values, 1):
    print(f"f_{i} = {fk}")


f_1 = 0.16666666666666666
f_2 = 0.2222222222222222
f_3 = 0.24074074074074073
f_4 = 0.24691358024691357
f_5 = 0.24897119341563786
f_6 = 0.2496570644718793
f_7 = 0.2498856881572931
f_8 = 0.24996189605243102
f_9 = 0.24998729868414368
f_10 = 0.2499957662280479
f_11 = 0.24999858874268263
f_12 = 0.24999952958089422
f_13 = 0.24999984319363142
f_14 = 0.24999994773121048
f_15 = 0.24999998257707015
f_16 = 0.24999999419235672
f_17 = 0.2499999980641189
f_18 = 0.2499999993547063
f_19 = 0.2499999997849021
f_20 = 0.2499999999283007
f_21 = 0.24999999997610023
f_22 = 0.2499999999920334
f_23 = 0.24999999999734446
f_24 = 0.24999999999911482
f_25 = 0.24999999999970493
f_26 = 0.24999999999990163
f_27 = 0.24999999999996722
f_28 = 0.2499999999999891
f_29 = 0.24999999999999636
f_30 = 0.24999999999999878
f_31 = 0.24999999999999958
f_32 = 0.24999999999999986
f_33 = 0.24999999999999997
f_34 = 0.25
f_35 = 0.25
f_36 = 0.25
f_37 = 0.25
f_38 = 0.25
f_39 = 0.25
f_40 = 0.25
f_41 = 0.25
f_42 = 0.25
f_43 = 0.25
f_44 = 0

In [7]:
f1 = 0.25

iterations = 100

fk_values = [f1]

for k in range(1, iterations):
    fk_next = (fk_values[-1] + 0.5) / 2
    fk_values.append(fk_next)

for i, fk in enumerate(fk_values, 1):
    print(f"f_{i} = {fk}")


f_1 = 0.25
f_2 = 0.375
f_3 = 0.4375
f_4 = 0.46875
f_5 = 0.484375
f_6 = 0.4921875
f_7 = 0.49609375
f_8 = 0.498046875
f_9 = 0.4990234375
f_10 = 0.49951171875
f_11 = 0.499755859375
f_12 = 0.4998779296875
f_13 = 0.49993896484375
f_14 = 0.499969482421875
f_15 = 0.4999847412109375
f_16 = 0.49999237060546875
f_17 = 0.4999961853027344
f_18 = 0.4999980926513672
f_19 = 0.4999990463256836
f_20 = 0.4999995231628418
f_21 = 0.4999997615814209
f_22 = 0.49999988079071045
f_23 = 0.4999999403953552
f_24 = 0.4999999701976776
f_25 = 0.4999999850988388
f_26 = 0.4999999925494194
f_27 = 0.4999999962747097
f_28 = 0.49999999813735485
f_29 = 0.4999999990686774
f_30 = 0.4999999995343387
f_31 = 0.49999999976716936
f_32 = 0.4999999998835847
f_33 = 0.49999999994179234
f_34 = 0.49999999997089617
f_35 = 0.4999999999854481
f_36 = 0.49999999999272404
f_37 = 0.499999999996362
f_38 = 0.499999999998181
f_39 = 0.4999999999990905
f_40 = 0.49999999999954525
f_41 = 0.4999999999997726
f_42 = 0.4999999999998863
f_43 = 0.4999999

In [8]:
f1 = 1/6
n = 50

iterations = 100

fk_values = [f1]

for k in range(1, iterations):
    fk_next = ((n-2)*fk_values[-1] + 0.5) / n
    fk_values.append(fk_next)

for i, fk in enumerate(fk_values, 1):
    print(f"f_{i} = {fk}")


f_1 = 0.16666666666666666
f_2 = 0.17
f_3 = 0.1732
f_4 = 0.17627199999999998
f_5 = 0.17922111999999998
f_6 = 0.18205227519999997
f_7 = 0.18477018419199998
f_8 = 0.18737937682432
f_9 = 0.1898842017513472
f_10 = 0.19228883368129332
f_11 = 0.19459728033404158
f_12 = 0.19681338912067992
f_13 = 0.19894085355585273
f_14 = 0.2009832194136186
f_15 = 0.20294389063707385
f_16 = 0.2048261350115909
f_17 = 0.20663308961112725
f_18 = 0.20836776602668217
f_19 = 0.21003305538561487
f_20 = 0.21163173317019027
f_21 = 0.21316646384338267
f_22 = 0.21463980528964735
f_23 = 0.21605421307806147
f_24 = 0.217412044554939
f_25 = 0.21871556277274146
f_26 = 0.21996694026183178
f_27 = 0.2211682626513585
f_28 = 0.22232153214530417
f_29 = 0.22342867085949203
f_30 = 0.22449152402511235
f_31 = 0.22551186306410784
f_32 = 0.22649138854154352
f_33 = 0.22743173299988179
f_34 = 0.2283344636798865
f_35 = 0.22920108513269102
f_36 = 0.2300330417273834
f_37 = 0.23083172005828806
f_38 = 0.23159845125595652
f_39 = 0.2323345132057

In [5]:
f1 = 0.1 

iterations = 100

fk_values = [f1]

for k in range(1, iterations):
    fk_next = (3 * fk_values[-1] + 0.5) / 5
    fk_values.append(fk_next)

for i, fk in enumerate(fk_values, 1):
    print(f"f_{i} = {fk}")


f_1 = 0.1
f_2 = 0.16
f_3 = 0.196
f_4 = 0.21760000000000002
f_5 = 0.23056000000000001
f_6 = 0.23833600000000002
f_7 = 0.2430016
f_8 = 0.24580096
f_9 = 0.247480576
f_10 = 0.2484883456
f_11 = 0.24909300736
f_12 = 0.24945580441600002
f_13 = 0.24967348264960001
f_14 = 0.24980408958976003
f_15 = 0.24988245375385604
f_16 = 0.24992947225231363
f_17 = 0.24995768335138818
f_18 = 0.24997461001083293
f_19 = 0.24998476600649977
f_20 = 0.24999085960389986
f_21 = 0.24999451576233991
f_22 = 0.24999670945740396
f_23 = 0.24999802567444238
f_24 = 0.24999881540466543
f_25 = 0.24999928924279927
f_26 = 0.24999957354567956
f_27 = 0.24999974412740772
f_28 = 0.24999984647644463
f_29 = 0.2499999078858668
f_30 = 0.24999994473152007
f_31 = 0.24999996683891204
f_32 = 0.24999998010334723
f_33 = 0.24999998806200835
f_34 = 0.24999999283720503
f_35 = 0.24999999570232304
f_36 = 0.24999999742139384
f_37 = 0.24999999845283633
f_38 = 0.2499999990717018
f_39 = 0.2499999994430211
f_40 = 0.24999999966581266
f_41 = 0.24999999

# Code for extension

In [9]:
n = 15
x = 0.5/n
print(n, x)

for k in range(int(n/2-1)):
    n -= 2
    x = (0.5 + n*x)/n
    print(n, x)

15 0.03333333333333333
13 0.07179487179487179
11 0.11724941724941723
9 0.17280497280497278
7 0.2442335442335442
5 0.3442335442335442
3 0.5109002109002109
